In [0]:
# Define variables
scope = "izanabees-kv"
key = "KEY-ADLS"
storage_account_name = "izanabees"
container_name = "bees"
mount_point = '/mnt/bees'

# Check if mount point is already mounted
if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
  
  # Get the connection string from secrets
  connect_str = dbutils.secrets.get(scope=scope, key=key)

  # Mount the container
  dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point=mount_point,
    extra_configs={
      f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": connect_str
    }
  )
  print(f"Mounted {mount_point}")
else:
  print(f"{mount_point} is already mounted")


/mnt/bees is already mounted


In [0]:
%fs ls /mnt/bees

path,name,size,modificationTime
dbfs:/mnt/bees/bronze/,bronze/,0,0
dbfs:/mnt/bees/gold/,gold/,0,0
dbfs:/mnt/bees/silver/,silver/,0,0


In [0]:
df_silver = spark.read.json("/mnt/bees/bronze/breweries.json")


In [0]:
df_silver.write.partitionBy('state').mode('overwrite').parquet('/mnt/bees/silver/breweries_silver')